<h1><center>Darwin Supervised Regression Model Building </center></h1>

First, 
<br>if you have just received a new api key from support, you will need to register your key and create a new user (see Register user cell)

Second, in the Environment Variables cell: 
1. Set your username and password to ensure that you're able to log in successfully
2. Set the path to the location of your datasets if you are using your own data.  The path is set for the examples.
3. Set the dataset names accordingly

Here are a few things to be mindful of:
1. For every run, check the job status (i.e. requested, failed, running, completed) and wait for job to complete before proceeding. 
2. If you're not satisfied with your model and think that Darwin can benefit from extra training, use the resume function.

### Set Darwin SDK

In [4]:
from amb_sdk.sdk import DarwinSdk
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/');

(True, 'https://amb-demo-api.sparkcognition.com/v1/')

### Environment Variables

In [5]:
#Set your user id and password accordingly
USER="justin.kang23@gmail.com"
PW="psjf52vRYb"

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET='data/'
DATASET_NAME='data.csv'

# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

### Import necessary libraries

In [6]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import r2_score

### User Login 

In [7]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

### Data Upload

**Read dataset and view a file snippet**

In [43]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, 'exemptions.csv'))
df.drop(df.tail(1).index,inplace=True)


for file in os.listdir(PATH_TO_DATASET):
    if 'csv' in file and 'exemptions' not in file: #dont add column twice
        print(file)
        df2=pd.read_csv(os.path.join(PATH_TO_DATASET, file)).iloc[1:]
        df=df.merge(df2, on='County')
df.head()

age.csv
education.csv
ethnicity.csv
financial.csv
general.csv
income.csv
population.csv
tax.csv


,County,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,Median Age,...,County Population,Population Density Per Sq Mile,County Seat,County Seat Population,General Fund M&O Tax Rate,General Fund I&S Tax Rate,Special Road & Bridge Fund M&O Tax Rate,Special Road & Bridge Fund I&S Tax Rate,Farm-to-Market/Flood Control Fund M&O Tax Rate,Farm-to-Market/Flood Control I&S Tax Rate
0,Andrews,0.36%,0.52%,0.63%,0.66%,0.82%,0.75%,0.90%,1.07%,31.8,...,"17,722",9.85,Andrews,"11,088",$0.438400,$0.000000,$0.077300,$0.000000,$0.085000,$0.000000
1,Angelina,0.33%,0.44%,0.46%,0.55%,0.49%,0.70%,0.63%,0.67%,37.1,...,"87,805",108.77,Lufkin,"35,067",$0.369130,$0.020480,$0.069190,$0.000000,$0.000000,$0.000000
2,Aransas,0.25%,0.46%,0.52%,0.98%,1.20%,1.22%,1.39%,1.49%,49.3,...,"25,572",91.87,Rockport,"8,766",$0.281598,$0.052278,$0.000000,$0.000000,$0.055222,$0.014217
3,Archer,0.60%,0.49%,0.61%,0.33%,0.56%,0.61%,0.45%,0.81%,44.3,...,"8,809",10.03,Archer City,"1,834",$0.457730,$0.071500,$0.046380,$0.038110,$0.061310,$0.000000
4,Armstrong,0.59%,1.43%,1.44%,1.98%,2.06%,0.59%,1.90%,1.44%,45.2,...,"1,879",2.09,Claude,"1,196",$0.444128,$0.000000,$0.000000,$0.000000,$0.218671,$0.000000


**Upload dataset to Darwin**

In [12]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, DATASET_NAME))
if not status:
    print(dataset)

**Clean dataset**

In [ ]:
# clean dataset
target = "Assessed_Value"
status, job_id = ds.clean_data(DATASET_NAME, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

### Create and Train Model 

We will now build a model that will learn the class labels in the target column.<br> In the default boston dataset, the target column is "Assessed_Value". <br> You will have to specify your own target name for your custom dataset. <br> You can also increase max_train_time for longer training.


In [ ]:
model = target + "_model0" + ts
status, job_id = ds.create_model(dataset_names = DATASET_NAME, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

### Extra Training (Optional)
Run the following cell for extra training, no need to specify parameters

In [ ]:
# Train some more
status, job_id = ds.resume_training_model(dataset_names = DATASET_NAME,
                                          model_name = model,
                                          max_train_time = '00:05')
                                          
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

### Analyze Model
Analyze model provides feature importance ranked by the model. <br> It indicates a general view of which features pose a bigger impact on the model

In [ ]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

Show the 10 most important features of the model.

In [ ]:
feature_importance[:10]

### Predictions
**Perform model prediction on the the training dataset.**

In [ ]:
status, artifact = ds.run_model(DATASET_NAME, model)
sleep(1)
ds.wait_for_job(artifact['job_name'])

Download predictions from Darwin's server.

In [ ]:
status, prediction = ds.download_artifact(artifact['artifact_name'])
prediction.head()

Create plots comparing predictions with actual target

In [ ]:
#Plot predictions vs actual
plt.plot(df[target], prediction[target], '.')
plt.plot([0,2.3e7],[0,2.3e7],'--k')
print('R^2 : ', r2_score(df[target], prediction[target]))

### Find out which machine learning model did Darwin use:

In [ ]:
status, model_type = ds.lookup_model_name(model)
print(model_type['description']['best_genome'])